In [29]:
# Bibliotecas
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

# Carregar dados

In [2]:
# Textos das proposicoes
proposicoes = list(pd.read_csv("../dados/proposicoes.csv")['texto'])

# Normalização de texto

In [3]:
nlp = spacy.load("pt_core_news_sm")

In [4]:
# Tokenização dos textos
textos_tokenizados = []
for texto in proposicoes:
    texto_tokenizado = nlp(texto)
    textos_tokenizados.append(texto_tokenizado)

In [5]:
# Identificação de partes da fala
textos_partes_fala = []
for tt in textos_tokenizados:
    textos_partes_fala.append([(p.text, p.pos_) for p in tt])

In [6]:
# Filtragem de partes da fala de interesse:
# NOUN, PROPN, ADJ, VERB
proposicoes_filtradas = []
for t in textos_partes_fala:
    texto_filtrado = []
    for palavra, parte_fala in t:
        if parte_fala in ['NOUN', 'PROPN', 'ADJ', 'VERB']:
            texto_filtrado.append(palavra)
    proposicoes_filtradas.append(" ".join(texto_filtrado).lower())

# Modelagem de tópicos

In [19]:
# Vetorizador TF (frequencia de termos nos documentos)
vetorizador_tf = CountVectorizer(max_df=0.5, min_df=3)
tf_textos = vetorizador_tf.fit_transform(proposicoes_filtradas)

In [24]:
# Número de tópicos
K = 20
alfa = 0.01
beta = 0.001
lda = LatentDirichletAllocation(n_components=20, max_iter=100, doc_topic_prior=alfa,
                               topic_word_prior=beta, n_jobs=7, verbose=1)
lda.fit(tf_textos)

iteration: 1 of max_iter: 100
iteration: 2 of max_iter: 100
iteration: 3 of max_iter: 100
iteration: 4 of max_iter: 100
iteration: 5 of max_iter: 100
iteration: 6 of max_iter: 100
iteration: 7 of max_iter: 100
iteration: 8 of max_iter: 100
iteration: 9 of max_iter: 100
iteration: 10 of max_iter: 100
iteration: 11 of max_iter: 100
iteration: 12 of max_iter: 100
iteration: 13 of max_iter: 100
iteration: 14 of max_iter: 100
iteration: 15 of max_iter: 100
iteration: 16 of max_iter: 100
iteration: 17 of max_iter: 100
iteration: 18 of max_iter: 100
iteration: 19 of max_iter: 100
iteration: 20 of max_iter: 100
iteration: 21 of max_iter: 100
iteration: 22 of max_iter: 100
iteration: 23 of max_iter: 100
iteration: 24 of max_iter: 100
iteration: 25 of max_iter: 100
iteration: 26 of max_iter: 100
iteration: 27 of max_iter: 100
iteration: 28 of max_iter: 100
iteration: 29 of max_iter: 100
iteration: 30 of max_iter: 100
iteration: 31 of max_iter: 100
iteration: 32 of max_iter: 100
iteration: 33 of 

LatentDirichletAllocation(batch_size=128, doc_topic_prior=0.01,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='batch', learning_offset=10.0,
             max_doc_update_iter=100, max_iter=100, mean_change_tol=0.001,
             n_components=20, n_jobs=7, n_topics=None, perp_tol=0.1,
             random_state=None, topic_word_prior=0.001,
             total_samples=1000000.0, verbose=1)

In [36]:
def imprimir_top_palavras(lda, palavras, topn):
    probabilidades =\
        lda.components_ / lda.components_.sum(axis=1)[:, np.newaxis]
    for topico_idx, topico in enumerate(probabilidades):
        top_palavras_idx = topico.argsort()[:-topn - 1:-1]
        print("Tópico %d:" % topico_idx)
        for i in top_palavras_idx:
            print("        " + palavras[i] + "   " + str(round(topico[i], 6)))
    print()

In [37]:
imprimir_top_palavras(lda, vetorizador_tf.get_feature_names(), 10)

Tópico 0:
        saúde   0.04928
        dispõe   0.045736
        sistema   0.028377
        dos   0.022442
        exercício   0.019891
        único   0.017399
        setembro   0.016555
        das   0.013476
        condições   0.013035
        estado   0.012727
Tópico 1:
        nacional   0.045662
        institui   0.038655
        dos   0.033793
        dispõe   0.027103
        dia   0.023233
        dezembro   0.021411
        programa   0.018769
        produção   0.01354
        agosto   0.013019
        outubro   0.012928
Tópico 2:
        empresas   0.02588
        novembro   0.025191
        público   0.023056
        poder   0.019526
        abril   0.01858
        empresa   0.01822
        dezembro   0.017489
        das   0.015586
        serviços   0.014259
        procedimento   0.013632
Tópico 3:
        dos   0.034206
        saúde   0.026415
        crimes   0.019872
        privados   0.014348
        junho   0.013263
        rol   0.013147
        julho   0.